<a href="https://colab.research.google.com/github/lisosoma/BI_ML/blob/main/HW3_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Деревья решений решают проблемы
__Суммарное количество баллов: 10__

Вы уже знакомы с классификацией методом KNN. В этом задании предстоит реализовать другой метод классификации - дерево решений. 

Одной из его особенностей является возможность объяснить в человекочитаемой форме, почему мы отнесли объект к определенному классу. Эта особенность позволяет использовать деревья решений для создания систем, которые могут подсказывать специалистам, на что именно стоит обратить внимание при принятии решений.

In [ ]:
from sklearn.datasets import make_blobs, make_moons
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm

## Задание 1 (1 балл)

Во время построения дерева решений нам потребуется определить, какой из предикатов лучше всего разбивает обучающую выборку. Есть два критерия, которые позволяют это сделать: критерий Джини и энтропийный критерий. Первый для подсчета информативности разбиения использует коэффициент Джини, второй - энтропию. Реализуйте подсчет этих коэффициентов, а так же подсчет информативности разбиения. 

#### Описание функций
`gini(x)` считает коэффициент Джини для массива меток

`entropy(x)` считает энтропию для массива меток

`gain(left_y, right_y, criterion)` считает информативность разбиения массива меток на левую `left_y` и правую `right_y` части при помощи `criterion`, который задается функцией (не строкой).

In [ ]:
def gini(x):
    classes = x.values()
    return 1 - sum([(n / sum(classes)) ** 2 for n in classes])

def entropy(x):
    classes = x.values()
    return -sum([n / sum(classes) * np.log2(n / sum(classes)) for n in classes])

def gain(lefty, righty, criterion):
    left, right = Counter(lefty), Counter(righty)
    n = left + right
    
    return (criterion(n) - sum(left) * criterion(left) / 
            (sum(left) + sum(right) + 1) - sum(right)
            * criterion(right) / (sum(left) + sum(right) + 1))

## Задание 2 (1 балл)

Деревья решений имеют хорошую интерпретируемость, т.к. позволяют не только предсказать класс, но и объяснить, почему мы предсказали именно его. Например, мы можем его нарисовать. Чтобы сделать это, нам необходимо знать, как оно устроено внутри. Реализуйте классы, которые будут задавать структуру дерева. 

#### DecisionTreeLeaf
Поля:
1. `y` должно содержать класс, который встречается чаще всего среди элементов листа дерева

#### DecisionTreeNode
В данной домашней работе мы ограничемся порядковыми и количественными признаками, поэтому достаточно хранить измерение и значение признака, по которому разбиваем обучающую выборку.

Поля:
1. `split_dim` измерение, по которому разбиваем выборку
2. `split_value` значение, по которому разбираем выборку
3. `left` поддерево, отвечающее за случай `x[split_dim] < split_value`. Может быть `DecisionTreeNode` или `DecisionTreeLeaf`
4. `right` поддерево, отвечающее за случай `x[split_dim] >= split_value`. Может быть `DecisionTreeNode` или `DecisionTreeLeaf`

__Интерфейс классов можно и нужно менять при необходимости__ (например, для вычисления вероятности в следующем задании)

In [ ]:
class DecisionTreeLeaf:
    def __init__(self, y):
        self.y = Counter(y).most_common(1)[0][0]
        self.data = y

class DecisionTreeNode:
    def __init__(self, left, right, split_dim, split_value, idx):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right
        self.idx = idx

## Задание 3 (6 баллов)

Теперь перейдем к самому дереву решений. Реализуйте класс `DecisionTreeClassifier`.

#### Описание методов
`fit(X, y)` строит дерево решений по обучающей выборке.

`predict_proba(X)` для каждого элемента из `X` возвращает словарь `dict`, состоящий из пар `(класс, вероятность)`. Вероятности классов в листе можно определить через количество объектов соответствующего класса в листе. 

#### Описание параметров конструктора
`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

#### Описание полей
`root` - корень дерева. Может быть `DecisionTreeNode` или `DecisionTreeLeaf`

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, criterion=None, max_depth=None, min_samples_leaf=30):
        self.root = None
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
    
    # функция для нахождения наилучших параметров
    def best(self, X):
        scores = []
        
        for col in tqdm(X.columns):
            impur, spvalue = 0, 0
            
            if col != 'Expected':
                
                # будем перебирать только уникальные значения из столбца
                for elem in set(X[col]):
                    leftX = X[X[col] <= elem]
                    rightX = X[X[col] > elem]
                    current = gain(leftX['Expected'], rightX['Expected'], 
                                            self.criterion)
                    if current > impur:
                        impur, spvalue = current, elem
                        
                scores.append([impur, spvalue, col])
                
        impur, spvalue, spdim = max(scores, key=lambda x: x[0])
        
        return impur, spvalue, spdim
    
    # функция для создания дерева
    # так как глубина дерева может быть любой
    # данную функцию нужно будет рекурсивно вызывать
    # рекурсия завершается лишь в том случае
    # когда глубина дерева достигает максимальной
    
    def tree(self, X, y, depth_tree):
        X['Expected'] = y
        
        # если достигли максимальной глубины, то прерываем рекурсию
        if depth_tree == self.max_depth:
            return DecisionTreeLeaf(y)
        
        impur, spvalue, spdim = self.best(X)
        
        xleft = X[X[spdim] <= spvalue].drop('Expected', axis=1)
        xright = X[X[spdim] > spvalue].drop('Expected', axis = 1)
        yleft = X[X[spdim] <= spvalue]['Expected']
        yright = X[X[spdim] > spvalue]['Expected']
        
        if len(yleft) < self.min_samples_leaf or len(yright) < self.min_samples_leaf:
            return DecisionTreeLeaf(y)
        
        left = self.tree(xleft, yleft, depth_tree + 1)
        right = self.tree(xright, yright, depth_tree + 1)
        
        return DecisionTreeNode(left, right, spdim, spvalue, impur)
    
    def fit(self, X, y):
        self.root = self.tree(X, y, 0)
    
    def predict_proba(self, X):
        ans = []
        
        for idx, x in X.iterrows():
            root = self.root
            
            while isinstance(root, DecisionTreeNode):
                if x[root.split_dim] <= root.split_value:
                    root = root.left
                else:
                    root = root.right
                    
            buf = Counter(root.data)
            
            ans.append({0 : buf[0] / (buf[0] + buf[1]), 1 : buf[1] / (buf[0] + buf[1])})
        return ans
    
    def predict(self, X):
        prob = self.predict_proba(X)
        
        return [max(p.keys(), key=lambda k: p[k]) for p in prob]

Построенное дерево можно нарисовать. Метод `draw_tree` рисует дерево и сохраняет его в указанный файл.

In [ ]:
def tree_depth(tree_root):
    if isinstance(tree_root, DecisionTreeNode):
        return max(tree_depth(tree_root.left), tree_depth(tree_root.right)) + 1
    else:
        return 1

def draw_tree_rec(tree_root, x_left, x_right, y):
    x_center = (x_right - x_left) / 2 + x_left
    if isinstance(tree_root, DecisionTreeNode):
        x_center = (x_right - x_left) / 2 + x_left
        x = draw_tree_rec(tree_root.left, x_left, x_center, y - 1)
        plt.plot((x_center, x), (y - 0.1, y - 0.9), c=(0, 0, 0))
        x = draw_tree_rec(tree_root.right, x_center, x_right, y - 1)
        plt.plot((x_center, x), (y - 0.1, y - 0.9), c=(0, 0, 0))
        plt.text(x_center, y, "x[{}] < {}".format(tree_root.split_dim, tree_root.split_value),
                horizontalalignment='center')
    else:
        plt.text(x_center, y, str(tree_root.y),
                horizontalalignment='center')
    return x_center

def draw_tree(tree, save_path=None):
    td = tree_depth(tree.root)
    plt.figure(figsize=(0.33 * 2 ** td, 2 * td))
    plt.xlim(-1, 1)
    plt.ylim(0.95, td + 0.05)
    plt.axis('off')
    draw_tree_rec(tree.root, -1, 1, td)
    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path)
    plt.show()

Для двумерного набора данных дерево можно отобразить на плоскости с данными. Кроме того, как и для любого классификатора, для него можно построить roc-кривую

In [ ]:

def plot_roc_curve(y_test, p_pred):
    positive_samples = sum(1 for y in y_test if y == 0)
    tpr = []
    fpr = []
    for w in np.arange(-0.01, 1.02, 0.01):
        y_pred = [(0 if p.get(0, 0) > w else 1) for p in p_pred]
        tpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt == 0) / positive_samples)
        fpr.append(sum(1 for yp, yt in zip(y_pred, y_test) if yp == 0 and yt != 0) / (len(y_test) - positive_samples))
    plt.figure(figsize = (7, 7))
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.xlim(-0.01, 1.01)
    plt.ylim(-0.01, 1.01)
    plt.tight_layout()
    plt.show()

def rectangle_bounds(bounds):
    return ((bounds[0][0], bounds[0][0], bounds[0][1], bounds[0][1]), 
            (bounds[1][0], bounds[1][1], bounds[1][1], bounds[1][0]))

def plot_2d_tree(tree_root, bounds, colors):
    if isinstance(tree_root, DecisionTreeNode):
        if tree_root.split_dim:
            plot_2d_tree(tree_root.left, [bounds[0], [bounds[1][0], tree_root.split_value]], colors)
            plot_2d_tree(tree_root.right, [bounds[0], [tree_root.split_value, bounds[1][1]]], colors)
            plt.plot(bounds[0], (tree_root.split_value, tree_root.split_value), c=(0, 0, 0))
        else:
            plot_2d_tree(tree_root.left, [[bounds[0][0], tree_root.split_value], bounds[1]], colors)
            plot_2d_tree(tree_root.right, [[tree_root.split_value, bounds[0][1]], bounds[1]], colors)
            plt.plot((tree_root.split_value, tree_root.split_value), bounds[1], c=(0, 0, 0))
    else:
        x, y = rectangle_bounds(bounds)
        plt.fill(x, y, c=colors[tree_root.y] + [0.2])

def plot_2d(tree, X, y):
    plt.figure(figsize=(9, 9))
    colors = dict((c, list(np.random.random(3))) for c in np.unique(y))
    bounds = list(zip(np.min(X, axis=0), np.max(X, axis=0)))
    plt.xlim(*bounds[0])
    plt.ylim(*bounds[1])
    plot_2d_tree(tree.root, list(zip(np.min(X, axis=0), np.max(X, axis=0))), colors)
    for c in np.unique(y):
        plt.scatter(X[y==c, 0], X[y==c, 1], c=[colors[c]], label=c)
    plt.legend()
    plt.tight_layout()
    plt.show()

## Задание 4 (2 балла)

Протестируйте решение на датасете spam.
Для этой задачи используйте данные x_spam_train и y_spam_train:
1. Выполните загрузку и предобработку файлов x_spam_train и y_spam_train.
2. Разбейте x_spam_train и y_spam_train на x_train, y_train, x_test и y_test для оценки точности работы алгоритма.
3. Посчитайте метрики `precision`, `recall`, `accuracy` для модели Decision Tree. Если необходимо, попробуйте разные наборы параметров для получения лучшего результата.
4. Сравните значения метрик с результатами модели kNN из предыдущего задания (можно использовать реализацию из библиотеки `sklearn`).
5. Ответьте на следующие вопросы:
    - Какой нужен препроцессинг данных для моделей?
    - Какая модель делает предсказания лучше?  Предположите, почему.

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/homework-decision-tree-ib-22/x_spam_train.csv')
X = df.drop(columns='Id')
y = pd.read_csv('/kaggle/input/homework-decision-tree-ib-22/y_spam_train.csv')
y = y['Expected']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95)
tree = DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, criterion=gini)
tree.fit(X_train, y_train)
plot_roc_curve(y_test, tree.predict_proba(X_test))
draw_tree(tree)

1. Обучите модель на всех данных из x_spam_train и y_spam_train.
2. Сделайте submit своего решения и получите значение f1_score не менее 0.6

In [ ]:
from sklearn.metrics import f1_score
y_pred = tree.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
X_spam_test = pd.read_csv('/kaggle/input/homework-decision-tree-ib-22/x_spam_test.csv')

In [ ]:
test = tree.predict(X_spam_test)

In [ ]:
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = X_spam_test['Id']
submission["Expected"] = test
submission.to_csv('submission.csv', index=False)

Метрики

In [ ]:
accuracy_score(y_test, y_pred) 

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
 precision_score(y_test, y_pred)

Сравнение метрик

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred) 

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

Как видно, у решающего дерева метрики лучше.

Лучше предсказания делает решающее дерево.

Для решающего дерева скорее всего нужна нормировка данных или избавление от столбцов, содержащих слишком много уникальных значений.

Для К-ближайших соседей тоже нужна нормировка.